In [1]:
import sqlite3
import json

In [6]:
# load in the sqlite file into memory

db_file= "scan_data.sqlite"

con = sqlite3.connect(db_file)
con.row_factory = sqlite3.Row
cur = con.cursor()

db_rows = con.execute(
    "SELECT ScanData.pkuid, ScanData.data, ScanData.level, \
    idx_ScanData_geometry.xmin, idx_ScanData_geometry.xmax, idx_ScanData_geometry.ymin, idx_ScanData_geometry.ymax \
    FROM ScanData \
    INNER JOIN idx_ScanData_geometry \
    ON ScanData.pkuid=idx_ScanData_geometry.pkid"
)
db_dict = [dict(row) for row in db_rows]

print(f"Obtained {len(db_dict)} rows from database.")

con.close()

Obtained 49 rows from database.


In [39]:
# turn sqlite file into dictionary of the following form

'''
    {
        geometry_tuple (): {
            level: int,
            scan_data: [],
        }
    }
'''

ap_dict = {}

for row in db_dict:
    x = (row['xmin'] + row['xmax']) / 2
    y = (row['ymin'] + row['ymax']) / 2
    
    level = row['level']
    
    if not level in ap_dict:
        ap_dict[level] = {}
    
    ap_dict[level][(x, y)] = row['data']
    
ap_dict = dict(sorted(ap_dict.items()))
print(f"Dict created, with {len(ap_dict)} keys with the following value lengths: \
{[len(val) for val in ap_dict.values()]}")


#for level, level_data in ap_dict.items():
#    print(f"Level {level}")
#    for geometry, scan in level_data.items():
#        print(f"({geometry}) := '<scan data omitted for readability>")

Dict created, with 6 keys with the following value lengths: [10, 10, 10, 10, 6, 3]
